In [1]:
# Run this cell to import all the libraries you need for this exercise
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat


In [2]:
# STEP 1: Read in data
data = loadmat('ex5data1.mat')

In [3]:
# Assuming m examples...
# X should be a (m x 400)-matrix, each row containing the 400 features of an example
# and y should be a (m x 1)-matrix containing the labels associated with the m examples

X = data['X']  
y = data['y']
print(X.shape, y.shape)

(5000, 400) (5000, 1)


In [20]:
# Turn y into a one-hot vector format
from sklearn.preprocessing import OneHotEncoder  
encoder = OneHotEncoder(sparse=False)  
y_onehot = encoder.fit_transform(y)

# print one of the y's out, say y[0]
print(y[0], y_onehot[0,:])

[10] [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [24]:
# standard sigmoid function
def sigmoid(z):  
    return 1 / (1 + np.exp(-z))

# Forward propagation
# Assuming:
#   input_size = 400  (number of neurons in input layer, also number of features for each example)
#   hidden_size = 25  (number of neurons in hidden layer)
#   num_labels = 10   (number of neurons in output layer)
# Inputs of the function:
#   X       (m x 400)-matrix   containing the input examples
#   theta1  (25 x 401)-matrix  containing the weights between the input layer of 401 neurons and the hidden layer of 25 neurons
#   theta2  (10 x 26)-matrix   containing the weights between the hidden layer of 25 neurons and the output layer of 10 neurons
# Outputs of the function
#   a1      (m x 401)-matrix   essentially X with an additional column of 1s inserted as column 0
#   z2      (m x 25)-matrix    contains a1 and theta1 multiplied together
#   a2      (m x 26)-matrix    apply sigmoid function to z2 with an additional column of 1s inserted as column 0
#   z3      (m x 10)-matrix    contains a2 and theta2 multiplied together
#   h       (m x 10)-matrix    contains the result of applying sigmoid to z3
def forward_propagate(X, theta1, theta2):  
    m = X.shape[0]
    a1 = np.hstack([ np.ones((m, 1)), X ])
    z2 = np.matmul(a1, np.transpose(theta1))
    a2 = np.hstack([ np.ones((m, 1)), sigmoid(z2) ])
    z3 = np.matmul(a2, np.transpose(theta2))
    h = sigmoid(z3)
    return a1, z2, a2, z3, h

# Cost function
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):  
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m
    
    # Add regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2))) 
    
    return J

In [25]:
# initial setup
input_size = 400  
hidden_size = 25  
num_labels = 10  
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = X.shape[0]  
X = np.matrix(X)  
y = np.matrix(y)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))  
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
print(theta1.shape, '', theta2.shape)


(25, 401)  (10, 26)


In [26]:
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)  
print(a1.shape, z2.shape, a2.shape, z3.shape, h.shape)


(5000, 401) (5000, 25) (5000, 26) (5000, 10) (5000, 10)


In [27]:
print(cost(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate))


7.062658812679908


In [28]:
def sigmoid_gradient(z):  
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):  
    ##### this section is identical to the cost function logic we already saw #####
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)

    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))

    ##### end of cost function logic, below is the new part #####

    # backpropagation
    # taking the examples one at a time
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)

        d3t = ht - yt  # (1, 10)

        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        
        #     d2t  (1 x 26)-matrix  containing deltas of the hidden layer
        d2t = np.multiply( np.matmul(d3t, theta2), sigmoid_gradient(z2t) )

        delta1 = delta1 + (d2t[:,1:]).T * a1t     # (25, 401)
        delta2 = delta2 + d3t.T * a2t             # (10, 26)

    delta1 = delta1 / m
    delta2 = delta2 / m

    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m

    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))

    return J, grad

In [29]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)  
print(J, grad.shape)


7.062658812679908 (10285,)


In [30]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate),  
                method='TNC', jac=True, options={'maxiter': 250})
print(fmin)


     fun: 0.3400304488143455
     jac: array([-9.71332903e-04,  1.15439338e-07, -1.19716163e-07, ...,
       -2.61070018e-05, -5.02251257e-05, -9.72975818e-05])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 22
  status: 3
 success: False
       x: array([ 8.32211238e-01,  5.77196689e-04, -5.98580816e-04, ...,
       -3.27946486e+00, -1.02354547e-01, -6.84224740e-02])


In [31]:
X = np.matrix(X)  
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))  
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)  
y_pred = np.array(np.argmax(h, axis=1) + 1)  
print(y_pred)


[[10]
 [10]
 [10]
 ...
 [ 9]
 [ 9]
 [ 9]]


In [32]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]  
accuracy = (sum(map(int, correct)) / float(len(correct)))  
print('accuracy = ', accuracy*100,'%')


accuracy =  99.32 %
